# 🩺 Healthcare Symptom Checker Agent

This notebook builds a simple AI agent that acts as a **healthcare assistant**.  
It asks clarifying questions, suggests common possible causes, and **always adds a disclaimer**:  

> ⚠️ This is not medical advice. Please consult a doctor.  

We will then extend this with frameworks like LangChain in Level 3.


## 🔹 Step 1: Install & Import Dependencies
We need:
- `openai` → to connect to GPT models  
- `python-dotenv` → to load our API key from a `.env` file  
- `ipykernel` → to run the notebook in VS Code  


In [2]:
#!pip install openai python-dotenv ipykernel --quiet


## 🔹 Step 2: Load API Key Safely

We need the API key to connect our healthcare agent with the OpenAI model.  
But we must keep it **private**.

### How we do it:
1. Store the key in a local `.env` file:

   OPENAI_API_KEY=sk-your-real-key-here

2. Load it in Python using the `dotenv` package.
3. This prevents us from writing the key directly in the notebook.

⚠️ Important: Never upload `.env` to GitHub. We’ll fix this later with `.gitignore`.


In [5]:


# Step 2: Load API Key safely

from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get the API key
api_key = os.getenv("OPENAI_API_KEY")

# Check if it's loaded
print("✅ API Key loaded:", "Yes" if api_key else "No")


✅ API Key loaded: Yes


## 🔹 Step 3: Define the Healthcare Agent

Our agent will:
1. Ask clarifying questions about the user’s symptoms.
2. Suggest possible general causes (non-diagnostic).
3. Remind the user this is **not medical advice** and recommend consulting a doctor.

### Type of Agent:
This is a **Goal-Based Agent** because:
- Goal = help users reason about symptoms and guide them safely.
- The agent takes actions (ask, suggest, remind) to achieve that goal.


In [1]:
# --------------------------------------
# 🔹 Step 3: Define the Healthcare Agent
# --------------------------------------

# Our agent will:
# 1. Ask clarifying questions about the user’s symptoms.
# 2. Suggest possible general causes (non-diagnostic).
# 3. Remind the user this is NOT medical advice and recommend consulting a doctor.

# ----------------------------------------------------
# Option A: REAL OpenAI GPT call (commented out)
# ----------------------------------------------------
# Uncomment if you have API credits & want to test live with GPT
"""
def healthcare_agent_openai(user_input):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful healthcare agent. \
             Ask clarifying questions, suggest possible causes of symptoms, \
             and remind the user this is not medical advice."},
            {"role": "user", "content": user_input}
        ]
    )
    return response.choices[0].message["content"]
"""

# ----------------------------------------------------
# Option B: SIMULATION MODE (safe & free)
# ----------------------------------------------------
def healthcare_agent(user_input):
    user_input = user_input.lower()

    # Simple rule-based responses
    if "headache" in user_input:
        return "It sounds like you have a headache. Common causes include dehydration, stress, or eye strain. Please consult a doctor if it’s severe."
    elif "fever" in user_input:
        return "A fever may be due to infections like flu or cold. Stay hydrated and rest, but seek medical help if it persists."
    elif "cough" in user_input:
        return "Coughs may be related to colds, flu, or allergies. Monitor your symptoms and consult a doctor if it lasts long."
    elif "bye" in user_input or "exit" in user_input:
        return "Take care! Remember, this is not medical advice. Please consult a healthcare professional for accurate diagnosis."
    else:
        return "I need more details about your symptoms. Could you describe what you’re feeling?"

# ----------------------------------------------------
# Run the agent (Simulation Mode)
# ----------------------------------------------------
print("Healthcare Agent: Hello! Tell me your symptoms (type 'bye' to exit).")

while True:
    user_input = input("You: ")
    response = healthcare_agent(user_input)
    print("Healthcare Agent:", response)
    if "bye" in user_input.lower() or "exit" in user_input.lower():
        break


Healthcare Agent: Hello! Tell me your symptoms (type 'bye' to exit).
Healthcare Agent: A fever may be due to infections like flu or cold. Stay hydrated and rest, but seek medical help if it persists.
Healthcare Agent: I need more details about your symptoms. Could you describe what you’re feeling?
Healthcare Agent: It sounds like you have a headache. Common causes include dehydration, stress, or eye strain. Please consult a doctor if it’s severe.
Healthcare Agent: Take care! Remember, this is not medical advice. Please consult a healthcare professional for accurate diagnosis.


# 🩺 Healthcare Symptom Checker Agent (Offline Simulation)

This notebook demonstrates an **improved Healthcare AI Agent** using **offline simulation**.  

The agent:
- Asks clarifying questions for vague symptoms
- Suggests common possible causes (non-diagnostic)
- Remembers previous inputs in the conversation
- Always reminds the user: ⚠️ **This is not medical advice**


## 🔹 Step 1: Install Required Packages

We will use:
- **langchain** → framework for agent logic
- **gradio** (optional) → simple interface


In [12]:
#!pip install langchain gradio


## 🔹 Why Offline Simulation?

- No API key required → fully free
- Safe for GitHub submission
- Simulates real agent behavior: perception → decision → action
- Can be extended later to use real LLMs


In [2]:
# Initialize conversation memory
conversation_history = []


## 🔹 Step 1: Define the Agent

The agent now:
- Uses **conversation history** to give context-aware responses
- Asks **clarifying questions** for vague input
- Handles multiple symptoms in one session


In [3]:
def healthcare_agent_improved(user_input):
    # Add user input to conversation history
    conversation_history.append(f"You: {user_input}")
    user_input_lower = user_input.lower()
    
    # Clarifying question for vague input
    if "not sure" in user_input_lower or "feel unwell" in user_input_lower or user_input_lower.strip() == "":
        response = "Can you describe your main symptom? Is it pain, fever, or cough?"
    
    # Symptom responses
    elif "headache" in user_input_lower:
        response = "It sounds like you have a headache. Common causes include dehydration, stress, or eye strain."
        # Context-aware advice
        if any("fever" in msg.lower() for msg in conversation_history):
            response += " Also, since you mentioned a fever earlier, consider consulting a doctor."
    
    elif "fever" in user_input_lower:
        response = "A fever may be caused by infections like flu or cold."
    
    elif "cough" in user_input_lower:
        response = "Coughs may be related to colds, flu, or allergies."
    
    # Exit condition
    elif "bye" in user_input_lower or "exit" in user_input_lower:
        response = "Take care! Remember, this is not medical advice."
    
    # Unknown input
    else:
        response = "I need more details about your symptoms. Can you describe them more clearly?"
    
    # Add agent response to history
    conversation_history.append(f"Agent: {response}")
    return response


## 🔹 Step 2: Interact with the Agent

Use a simple input/output loop in the notebook:
- Type symptoms
- Agent responds with advice and clarifying questions
- Type 'bye' or 'exit' to end the conversation


In [4]:
print("Healthcare Agent (Offline Simulation): Hello! Type 'bye' to exit.")

while True:
    user_input = input("You: ")
    response = healthcare_agent_improved(user_input)
    print("Healthcare Agent:", response)
    if "bye" in user_input.lower() or "exit" in user_input.lower():
        break


Healthcare Agent (Offline Simulation): Hello! Type 'bye' to exit.
Healthcare Agent: I need more details about your symptoms. Can you describe them more clearly?
Healthcare Agent: It sounds like you have a headache. Common causes include dehydration, stress, or eye strain.
Healthcare Agent: Take care! Remember, this is not medical advice.


## 🔹 Step 3: Optional GUI with Gradio

You can interact with the agent using a **web interface**:
- Input textbox for user symptoms
- Output textbox for agent response
- Works entirely offline


In [5]:
import gradio as gr

# Optional: keep a conversation history
conversation_history = []

def interact(symptoms):
    global conversation_history
    # Call your improved agent
    response = healthcare_agent_improved(symptoms)
    
    # Add to memory
    conversation_history.append(f"You: {symptoms}")
    conversation_history.append(f"Agent: {response}")
    
    # Display full conversation in output
    chat_output = "\n".join(conversation_history)
    return chat_output

iface = gr.Interface(
    fn=interact,
    inputs=gr.Textbox(lines=3, placeholder="Describe your symptoms here..."),
    outputs=gr.Textbox(lines=10, placeholder="Agent responses will appear here..."),
    title="Healthcare Symptom Checker (Offline)",
    description="Type your symptoms to get possible causes. ⚠️ This is not medical advice."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
